In [1]:
%cd /qbio/nest/alpaca/Tutorial

import numpy as np
import matplotlib.pyplot as plt
import math


#Generating dictoinary containing gene tags and their counts

def makedata(addr):

    with open(addr) as file_data:

        Result = {}
        for line in file_data:
            if line.startswith('@'):
                continue
            else:
                tmp1 = line.split('\t')
                if tmp1[1] == 4:
                    continue
                else:
                    tmp2 = tmp1[2].split('|')
                    tmp1[2] = tmp2[1]
                    if (tmp1[2] in Result):
                        Result[tmp1[2]] = Result[tmp1[2]] + 1
                    else:
                        Result[tmp1[2]] = 1
        return Result


/qbio/nest/alpaca/Tutorial


In [2]:
Rseq_untreated = makedata('Align_RNA-seq-untreated/Aligned.out.sam')
Rseq_siLuc = makedata('Align_RNA-seq-siLuc/Aligned.out.sam')
Rseq_siLin28a = makedata('Align_RNA-seq-siLin28a/Aligned.out.sam')
CLIP_Lin28a = makedata('Align_Lin28a-CLIP-35L33G/Aligned.out.sam')
RiboProf_siLin28a = makedata('Align_RiboProf-siLin28a/Aligned.out.sam')
RiboProf_siLuc = makedata('Align_RiboProf-siLuc/Aligned.out.sam')

In [7]:
cutoff = 250

#Calculating Lin28A CLIP enrichment

Xelements = {}

for a in CLIP_Lin28a:
    if a in Rseq_untreated:
        if (CLIP_Lin28a[a] > cutoff and Rseq_untreated[a] > cutoff):
            Xelements[a] = math.log(CLIP_Lin28a[a]/Rseq_untreated[a], 2)
    else:
        continue

#Calculating Ribosome density change upon Lin28a KO

tmp1 = {}
tmp2 = {}
Yelements = {}

for a in RiboProf_siLin28a:
    if a in Rseq_siLin28a:
        if (RiboProf_siLin28a[a] > cutoff and Rseq_siLin28a[a] > cutoff):
            tmp1[a] = RiboProf_siLin28a[a]/Rseq_siLin28a[a]
    else:
        continue

for a in RiboProf_siLuc:
    if a in Rseq_siLuc:
        if (RiboProf_siLuc[a] > cutoff and Rseq_siLuc[a] > cutoff):
            tmp2[a] = RiboProf_siLuc[a]/Rseq_siLuc[a]
    else:
        continue
        
for a in tmp1:
    if a in tmp2:
        Yelements[a] = math.log(tmp1[a]/tmp2[a], 2)
    else:
        continue


In [8]:
#Generating dictoinary containing gene tags and their location

with open('GO_Terms/uniprot-organism__mus+musculus_.tab') as file_data:

    CC = {}
    count = 0
    for line in file_data:
        if '(Mouse)' not in line:
            continue
        else:
            count += 1
            tmp1 = line.split('\t')
            tmp2 = tmp1[3].split('GO:')
            tmp3 = tmp1[2].split(';')
            Gene = []           
            location = []
            code = ''
            
            for i in range(len(tmp3)-1):
                Gene.append(tmp3[i][:18])
            
            if len(Gene)==0:
                continue
            
            for i in range(len(tmp2)-1):
                location.append(tmp2[i+1][:7])
                
            #We only need information whether given gene product is located in nucleus(GO:0005634), integral membrane(GO:0016021), and cytoplasm(GO:0005737).
            if '0005634' in location:
                code += '1'
            else:
                code += '0'
                
            if '0016021' in location:
                code += '1'
            else:
                code += '0'
                
            if '0005737' in location:
                code += '1'
            else:
                code += '0'
                
            code = str(code)
                
            
            for a in Gene:
                CC[a]=code
                
#Generate lists of data to plot

Nucleus = []
IntegralMembrane = []
Cytoplasm = []

print("size of Xelements: ")
print(len(Xelements))
print("size of Yelements: ")
print(len(Yelements))
print("size of CC: ")
print(len(CC))

for a in Xelements:
    print(a)
    break
    
for a in Yelements:
    print(a)
    break
    
for a in CC:
    print(a)
    break

for a in Xelements:
    if (a in Yelements and a[:len(a)-2] in CC):
        tmp = [Xelements[a], Yelements[a]]
        if CC[a[:-2]][0]=='1':
            Nucleus.append(tmp)
        if CC[a[:-2]][1]=='1':
            IntegralMembrane.append(tmp)
        if CC[a[:-2]][2]=='1':
            Cytoplasm.append(tmp)
    else:
        continue

Data_Nucleus = np.array(Nucleus)      
Data_IntegralMembrane = np.array(IntegralMembrane)
Data_Cytoplasm = np.array(Cytoplasm)

size of Xelements: 
4890
size of Yelements: 
8332
size of CC: 
67332
ENSMUSG00000106106.3
ENSMUSG00000106106.3
ENSMUST00000013773


In [9]:
#Plot figure S6.A
x, y = Data_Nucleus.T
plt.scatter(x,y,s=4,c='blue', label='Nucleus')
x, y = Data_IntegralMembrane.T
plt.scatter(x,y,s=4,c='red', label='Integral membrane')
x, y = Data_Cytoplasm.T
plt.scatter(x,y,s=4,c='green', label='Cytoplasm')

plt.title('Linkage to localization')
plt.xlabel('LIN28A CLIP enrichment(log_2)')
plt.ylabel('Ribosome density change upon Lin28a knockdown(log_2)')
plt.legend()
plt.grid()
plt.savefig('Figure S6A.png', dpi=500)
plt.show()

ValueError: not enough values to unpack (expected 2, got 0)